In [ ]:
# a module for loading MNIST dataset

import numpy as np
import torch

from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import random

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import matplotlib.pyplot as plt

import cv2

import os

import torchvision

import cv2
import PIL.Image


import os

import time

dtype = torch.float
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')





import my_coding




In [ ]:

# coding_layers should be a list of number of neurons in each layer 
# ones_in_layes should be a list of number of ones in each coding vector


def get_MNIST_dataset( coding_layers , ones_in_layes    , batch_size , device = device  , flatten = True ,  size = 28 , code_map = None):

  MNIST_original_train_dataset = datasets.MNIST(root='./mnist_data/', train=True , download=True  ,   transform=torchvision.transforms.Compose(
        [torchvision.transforms.Resize( size  ), torchvision.transforms.ToTensor()]
        
        ))
  
  MNIST_original_test_dataset = datasets.MNIST(root='./mnist_data/', train=False , download=True   ,  transform=torchvision.transforms.Compose(
        [torchvision.transforms.Resize( size  ), torchvision.transforms.ToTensor()]
        
        ))

  num_workers = 0



  if code_map==None :
    code_map = my_coding.get_n_hot_coding_map( coding_layers , ones_in_layes   , 10 )


  coded_train_labels = my_coding.code_category( code_map , MNIST_original_train_dataset.targets )

  coded_train_labels_cat = torch.cat( coded_train_labels  , dim=1  ) 


  coded_test_labels = my_coding.code_category( code_map , MNIST_original_test_dataset.targets  )
  coded_test_labels_cat = torch.cat( coded_test_labels  , dim=1  ) 


  
  x_train = MNIST_original_train_dataset.data / 255.0
 
  if size!=28:
    dataset_data_resized = torch.zeros([ len(MNIST_original_train_dataset) , size , size ]).to(dtype)

    for i in range( len(MNIST_original_train_dataset)   ):
      dataset_data_resized[i] = torch.tensor(  cv2.resize( x_train[i].numpy()  ,  ( int( size ) , int( size )  )  ) ).to(dtype)

    x_train = dataset_data_resized

  if (flatten == True ):
    x_train = x_train.reshape(  [ MNIST_original_train_dataset.data.shape[0] , -1 ]  ) .to(dtype)
  else:
    x_train = x_train.unsqueeze(  1)


  x_test = MNIST_original_test_dataset.data / 255.0

  if size!=28:
    dataset_data_resized = torch.zeros([ len(MNIST_original_test_dataset) , size , size ]).to(dtype)

    for i in range( len(MNIST_original_test_dataset)   ):
      dataset_data_resized[i] = torch.tensor(  cv2.resize( x_test[i].numpy()  ,  ( int( size ) , int( size )  )  ) ).to(dtype)

    x_test = dataset_data_resized

  if (flatten == True ):
    x_test = x_test.reshape(  [ MNIST_original_test_dataset.data.shape[0] , -1 ]  ) .to(dtype)
  else:
     x_test = x_test.unsqueeze(  1)

  # print("x_train.shape : " + str(x_train.shape) )
  # print("x_test.shape  : " + str(x_test.shape)  )

  # print("y_train.shape : " + str(coded_train_labels_cat.shape)  )
  # print("y_test.shape  : " + str(MNIST_original_test_dataset.targets.shape)  )



  # fig = plt.figure(num = None, figsize=(5,25)  , dpi=90, facecolor='w', edgecolor='k')
  # plt.subplot(1, 5, 1)
  # plt.imshow( x_train[0].view(size,size)  )

  # plt.axis('off')
  # plt.subplot(1, 5, 2)
  # plt.imshow( x_train[10].view(size,size) )
  # plt.axis('off')
  # plt.subplot(1, 5, 3)
  # plt.imshow( x_train[100].view(size,size)  )
  # plt.axis('off')
  # plt.subplot(1, 5, 4)
  # plt.imshow( x_train[99].view(size,size)  )
  # plt.axis('off')
  # plt.subplot(1, 5, 5)
  # plt.imshow( x_train[88].view(size,size)  )        
  # plt.axis('off')
  # plt.show()



  train_dataset = torch.utils.data.TensorDataset(   x_train.to('cpu')    , coded_train_labels_cat.to('cpu')     )
  test_dataset = torch.utils.data.TensorDataset(   x_test.to('cpu')  ,  MNIST_original_test_dataset.targets.to(dtype).to('cpu')    )

  train_loader = torch.utils.data.DataLoader( dataset=train_dataset , batch_size=batch_size , shuffle=False , num_workers=num_workers , pin_memory=True)
  test_loader = torch.utils.data.DataLoader(  dataset=test_dataset , batch_size=batch_size , shuffle=False , num_workers=num_workers ,pin_memory=True)

  inputs, classes = next(iter(train_loader))  

  # print("train loader x shape" + str( inputs.shape  )  )
  # print("train loader y shape" + str( classes.shape  )  )



  inputs, classes = next(iter(test_loader))  

  # print("test loader x shape" + str( inputs.shape  )  )
  # print("test loader y shape" + str( classes.shape  )  )


  x_shape = inputs.shape[1:]  
  return train_loader , test_loader , code_map , x_shape
